In [1]:
import os
%pwd

'f:\\kidney-Disease-Classification_DL\\research'

In [2]:
os.chdir("../")

In [3]:
%pwd

'f:\\kidney-Disease-Classification_DL'

In [4]:
os.environ['MLFLOW_TRACKING_URI'] = 'https://dagshub.com/shlok-dadhich/kidney-Disease-Classification_DL.mlflow'
os.environ['MLFLOW_TRACKING_USERNAME'] = 'shlok-dadhich'
os.environ['MLFLOW_TRACKING_PASSWORD'] = '585f29336580f8c0e6967749768d16c35060e4ca'

In [ ]:
import dagshub
dagshub.init(repo_owner='shlok-dadhich', repo_name='kidney-Disease-Classification_DL', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

In [5]:
import tensorflow as tf
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [8]:
from CNNClassifier.constants import *
from CNNClassifier.utils.common import read_yaml, create_directories,save_json

In [9]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = Config_PATH,
                 params_filepath = Params_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts\data_injection\CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone",
            mlflow_uri="https://dagshub.com/shlok-dadhich/kidney-Disease-Classification_DL.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config
    

    

In [ ]:
import tensorflow as tf
import mlflow
from pathlib import Path
import mlflow.keras
from urlib.parse import urlparse

In [11]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config
    
    def valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )
        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation="bilinear"
        )
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(**datagenerator_kwargs)
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(model_path: Path) -> tf.keras.Model:
        model = tf.keras.models.load_model(model_path)
        return model
    
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self.valid_generator()
        self.score = model.evaluate(self.valid_generator)

    def save_score(self):
        scores = {
            "loss": self.score[0],
            "accuracy": self.score[1]
        }
        save_json(path=Path("scores.json"), data=scores)
       
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics({
                "loss": self.score[0],
                "accuracy": self.score[1]
            })
            if tracking_url_type_store != "file":
                mlflow.keras.log_model(self.model, "model", registered_model_name="KidneyDiseaseModel")
            else:
                mlflow.keras.log_model(self.model, "model")

In [12]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()

    evaluation = Evaluation(config=eval_config)
    evaluation.evaluation()
    # evaluation.save_score()
    evaluation.log_into_mlflow()

except Exception as e:
    raise e

[2025-10-02 23:02:11,063: INFO: common]: yaml file: config\config.yaml loaded successfully
[2025-10-02 23:02:11,095: INFO: common]: yaml file: params.yaml loaded successfully
[2025-10-02 23:02:11,097: INFO: common]: Created directory at: artifacts
Found 2487 images belonging to 4 classes.
78/78 [==============================] - 766s 10s/step - loss: 1.1099 - accuracy: 0.5742


NameError: name 'urlparse' is not defined